# Extract URLs

> Extract all the relevant URLs you need to get that talks and metadata you would like to access.

In [ ]:
#| default_exp extract_URLs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from bs4 import BeautifulSoup
import requests
import re

def extract_talk_urls(
    url:str # The URL of a General Conference year page, e.g., 'https://www.churchofjesuschrist.org/study/general-conference/1994/04?lang=eng'
    )-> list[str]: # A list of URLs for talks from the given General Conference year page.
    
    response = requests.get(str(url))
    soup = BeautifulSoup(response.content, 'html.parser')

    # Define a regular expression pattern to match the desired URLs

    # for more recent talks
    url_pattern = re.compile(r'/study/general-conference/\d{4}/(04|10)/\d{2}[a-zA-Z]+.*\?lang=eng')

    # Find all <a> tags with an 'href' attribute that matches the pattern
    links = soup.find_all('a', href=url_pattern)
    
    # for older talks
    if len(links) == 0:
        url_pattern = re.compile(r'/study/general-conference/\d{4}/(04|10)/(?!.*session)[a-zA-Z]+.*\?lang=eng')
        links = soup.find_all('a', href=url_pattern)
    
    # Extract the URLs
    talk_urls = ['https://www.churchofjesuschrist.org/' + link['href'] for link in links]
    
    return talk_urls

In [ ]:
# As an example
url = "https://www.churchofjesuschrist.org/study/general-conference/1994/04?lang=eng"

all_talk_urls = extract_talk_urls(url)
print(all_talk_urls)

['https://www.churchofjesuschrist.org//study/general-conference/1994/04/five-loaves-and-two-fishes?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/to-be-healed?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/if-a-man-die-shall-he-live-again?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/a-childs-love-matured?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/walk-with-me?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/a-divine-prescription-for-spiritual-healing?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/the-father-and-the-family?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/the-sustaining-of-church-officers?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/1994/04/the-church-audit-committee-report?lang=eng', 'https://www.ch

In [ ]:
#| export
def generate_conference_url(
                            year:int, # The chosen year
                            month:str # The chosen month represented as a two-digit string (e.g., '04' for April, '10' for October).
                           ) -> str: # A General Conference year page URL corresponding to the specified year and month.
    if month in ['04', '10']:
        return f"https://www.churchofjesuschrist.org/study/general-conference/{year}/{month}?lang=eng"
    else:
        raise ValueError("Month must be either '04' (April) or '10' (October).")

In [ ]:
# As an example
print(generate_conference_url(2019, "10"))

https://www.churchofjesuschrist.org/study/general-conference/2019/10?lang=eng


In [ ]:
#| export
def generate_conference_urls(
                            start_year:int, # The start year for the range of years.
                            end_year:int|None=None, # The end year for the range of General years. Default is None (just produces the URLs for that year provided by start_year).
                            most_recent_april:bool=False # Decides whether to generate the URL for the October conference of that year depending on if the most recent April conference took place at the provided final year in the range. Default is False.
                            ) -> list[str]: #A list of General Conference year page URLs from the years within the specified range.
    conference_urls = []
    months = ['04', '10']
    if end_year is None:
        urls = [generate_conference_url(start_year, month) for month in months]
    else:
        urls = []
        for year in range(start_year, end_year+1, 1):
            year_urls = [generate_conference_url(year, month) for month in months]
            urls.append(year_urls)
        urls = [item for sublist in urls for item in sublist]
        
        if most_recent_april == True:
            urls = urls[:-1]
            
    return urls

In [ ]:
# As a first example
print("Show the URLs for just the year 2016:\n")
print(generate_conference_urls(2016))

print("\n\n")

# As a second example
print("Show the URLs for the years 2016 and 2017:\n")
print(generate_conference_urls(start_year=2016, end_year=2017, most_recent_april=True))

Show the URLs for just the year 2016:

['https://www.churchofjesuschrist.org/study/general-conference/2016/04?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2016/10?lang=eng']



Show the URLs for the years 2016 and 2017:

['https://www.churchofjesuschrist.org/study/general-conference/2016/04?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2016/10?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2017/04?lang=eng']


In [ ]:
#| export
def extract_multiconference_talk_urls(start_year:int, # Start year
                                end_year:int # End year
                               ) -> list[str]: # List of conference talk URLs
    gen_conf_page_urls = generate_conference_urls(start_year, end_year)

    multi_conference_talks = []
    
    # extract all the talks urls from all years
    for url in gen_conf_page_urls:
        multi_conference_talks.append(extract_talk_urls(url))
    
    # flatten the list
    multi_conference_talks = [item for sublist in multi_conference_talks for item in sublist]

    return multi_conference_talks

In [ ]:
# As an example for a single year (2017)
print(extract_multiconference_talk_urls(2017,2017))

['https://www.churchofjesuschrist.org//study/general-conference/2017/04/trust-in-the-lord-and-lean-not?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/the-beauty-of-holiness?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/certain-women?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/my-peace-i-leave-with-you?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/gathering-the-family-of-god?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/his-daily-guiding-hand?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/our-fathers-glorious-plan?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/our-good-shepherd?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/confide-in-god-unwaveringly?lang=eng', 'https://www.churchofjesuschrist.org//study/

In [ ]:
# As an example for a span of years (2017 to 2018
print(extract_multiconference_talk_urls(2017,2018))

['https://www.churchofjesuschrist.org//study/general-conference/2017/04/trust-in-the-lord-and-lean-not?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/the-beauty-of-holiness?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/certain-women?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/my-peace-i-leave-with-you?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/gathering-the-family-of-god?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/his-daily-guiding-hand?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/our-fathers-glorious-plan?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/our-good-shepherd?lang=eng', 'https://www.churchofjesuschrist.org//study/general-conference/2017/04/confide-in-god-unwaveringly?lang=eng', 'https://www.churchofjesuschrist.org//study/

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()